In [15]:
%load_ext autoreload
%autoreload 2

In [44]:
import pandas as pd
import re

from utils import get_data, Config

## Load Data + Preprocessing

In [57]:
# Load all data
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Response".id <= 116032;
"""
df = get_data(Config(), initial_data)

Connecting to the PostgreSQL database...
Connection successful


In [158]:
# Extra columns
df["outcome_str"] = df["outcome_value"].fillna("None").astype(str)

# Unify outcomes that are the same
# Firefox: {'error': 'object "TypeError: NetworkError when attempting to fetch resource."', 'headers': ''} Chromium: {'error': 'object "TypeError: Failed to fetch"', 'headers': ''}
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: NetworkError when attempting to fetch resource.\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")
# For document referrer we do not want to know the exact resp_id and count
# Maybe we even only want to know whether it is a origin or the full URl?
df['outcome_str'] = df['outcome_str'].replace(r'resp_id=\d+', 'resp_id=<resp_id>', regex=True)
df['outcome_str'] = df['outcome_str'].replace(r'count=\d+', 'count=<count>', regex=True)

## Overview

In [162]:
display(df.head(2))
display(df.describe())
display(df.columns)

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,status_code,label,resp_type,name,version,headless_mode,os,automation_mode,add_info,outcome_str
0,1,<class 'dict'>,"{'error': 'object ""TypeError: Failed to fetch""...",fetch_GET,0,None,None,https,sub.headers.websec.saarland,http,...,200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,,"{'error': 'object ""TypeError: Failed to fetch""..."
1,2,<class 'dict'>,"{'error': 'object ""TypeError: Failed to fetch""...",fetch_TEST,0,None,None,https,sub.headers.websec.saarland,http,...,200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,,"{'error': 'object ""TypeError: Failed to fetch""..."


,id,test_status,browser_id,testcase_id,response_id,created_at,updated_at,status_code
count,116032.000000,116032.000000,116032.000000,116032.0,116032.000000,116032,116032,116032.000000
mean,58016.500000,0.000052,52.621208,1.0,1396.614796,2023-11-17 20:35:29.695941120,2023-11-17 20:35:29.695942144,214.515168
min,1.000000,0.000000,13.000000,1.0,60.000000,2023-11-17 17:59:33.478706,2023-11-17 17:59:33.478712,200.000000
25%,29008.750000,0.000000,15.000000,1.0,342.000000,2023-11-17 19:15:53.122749696,2023-11-17 19:15:53.122755072,200.000000
50%,58016.500000,0.000000,71.000000,1.0,1370.000000,2023-11-17 20:37:59.985938944,2023-11-17 20:37:59.985939456,200.000000
75%,87024.250000,0.000000,73.000000,1.0,2082.000000,2023-11-17 21:56:40.352541440,2023-11-17 21:56:40.352542464,200.000000
max,116032.000000,2.000000,75.000000,1.0,3088.000000,2023-11-17 23:19:02.786333,2023-11-17 23:19:02.786335,302.000000
std,33495.697555,0.010169,25.793203,0.0,774.765090,NaN,NaN,35.635207


Index(['id', 'outcome_type', 'outcome_value', 'test_name', 'test_status',
       'test_message', 'test_stack', 'org_scheme', 'org_host', 'resp_scheme',
       'resp_host', 'relation_info', 'browser_id', 'testcase_id',
       'response_id', 'status', 'created_at', 'updated_at', 'raw_header',
       'status_code', 'label', 'resp_type', 'name', 'version', 'headless_mode',
       'os', 'automation_mode', 'add_info', 'outcome_str'],
      dtype='object')

In [71]:
# Collected results per browser
df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["id"].count().sort_values(ascending=False).to_frame()

id
name    version                   os           headless_mode automation_mode       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
chrome  119                       Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
edge    119                       Ubuntu 22.04 headless-new  selenium         11624
                                               xvfb          selenium         11624
firefox 119                       Ubuntu 22.04 xvfb          selenium         11528
                                               headless      selenium         11512

In [74]:
# Timeouts
display(df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["test_status"].value_counts().to_frame())

display(df.loc[df["test_status"] == 2])

count
name    version                   os           headless_mode automation_mode test_status       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11624
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11624
chrome  119                       Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11624
edge    119                       Ubuntu 22.04 headless-new  selenium        0            11624
                                               xvfb          selenium        0            11622
                                                                             2                2
firefox 119                       Ubuntu 22.04 headless      selenium        0            11512
                                               xvfb          selenium        0            11527
                                                                             2                1

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,raw_header,status_code,label,resp_type,name,version,headless_mode,os,automation_mode,add_info
73595,73631,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,"[[Referrer-Policy, no-referrer]]",200,RP,basic,firefox,119,xvfb,Ubuntu 22.04,selenium,
84995,84976,<class 'NoneType'>,None,oac_window.open,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,"[[origin-agent-cluster, ?0], [location, https:...",302,OAC,basic,edge,119,xvfb,Ubuntu 22.04,selenium,
85333,85352,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,"[[Referrer-Policy, unsafe-url], [location, htt...",302,RP,basic,edge,119,xvfb,Ubuntu 22.04,selenium,


In [80]:
# Time taken!
# Currently the HTTP times of Chromium-based browsers do not make too much sense due to HTTPS upgrades; however this is fixed for the next run
df.groupby(["name", "version", "os", "headless_mode", "automation_mode", "org_scheme"])["created_at"].agg(["min", "max"]).apply(lambda x: x["max"] - x["min"], axis=1).to_frame()

0
name    version                   os           headless_mode automation_mode org_scheme                       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        http       0 days 00:16:54.759901
                                                                             https      0 days 00:32:53.405975
                                               xvfb          selenium        http       0 days 00:16:57.517750
                                                                             https      0 days 00:32:59.348796
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        http       0 days 00:16:53.083071
                                                                             https      0 days 00:32:52.104873
                                               xvfb          selenium        http       0 days 00:16:58.926585
                                                                             https      0 days 00:33:05.305826
chrome  119                       Ubuntu 22.04 headless-new  selenium        http       0 days 00:00:49.338957
                                                                             https      0 days 00:32:25.278693
                                               xvfb          selenium        http       0 days 00:00:49.509877
                                                                             https      0 days 00:32:28.764379
edge    119                       Ubuntu 22.04 headless-new  selenium        http       0 days 00:12:11.420509
                                                                             https      0 days 00:16:02.139976
                                               xvfb          selenium        http       0 days 00:12:15.619910
                                                                             https      0 days 00:16:03.400237
firefox 119                       Ubuntu 22.04 headless      selenium        http       0 days 00:14:08.454610
                                                                             https      0 days 00:17:07.358566
                                               xvfb          selenium        http       0 days 00:15:20.420077
                                                                             https      0 days 00:18:06.464448

## Response analysis

- We have 11624 unique tests: (test_name x relation_info) x org_scheme x (resp_scheme x resp_host) x response_id)
- They should all have 10 (num browsers in various settings) outcomes
    -  Due to the HTTPS Upgrade bug many have either 16 or 4 outcomes only (as the 6 http outcomes of Chrome + Brave instead resulted in HTTPS outcomes)
    -  Without org_scheme they should have 20; a couple tests have less than 20 (due to the Firefox errors and/or timeouts?)

In [142]:
# Unique tests
tests = {
    "Fetch": [4, 2, 8, 6], 
    "Framing": [9, 2, 8, 35], # 7+11+17
    "PP": [4, 2, 8, 24],
    "OAC": [3, 2, 8, 14],
    "TAO": [1, 2, 8, 13],
    "RP": [2, 2, 8, 16],
    "CSP-SCRIPT": [2, 2, 8, 17],
    "COEP": [2, 2, 8, 11],
    "CORP": [4, 2, 8, 11],
    "CSP-IMG": [1, 2, 8, 17],
    "HSTS": [2, 1, 2, 14],
    "COOP": [1, 2, 8, 11],
}
tests = pd.DataFrame.from_dict(tests, columns=["tests", "org_origins", "resp_origins", "responses"], orient="index")
tests["All tests"] = tests.prod(axis=1)
tests.loc["Sum"] = tests.sum(axis=0)
tests

,tests,org_origins,resp_origins,responses,All tests
Fetch,4,2,8,6,384
Framing,9,2,8,35,5040
PP,4,2,8,24,1536
OAC,3,2,8,14,672
TAO,1,2,8,13,208
RP,2,2,8,16,512
CSP-SCRIPT,2,2,8,17,544
COEP,2,2,8,11,352
CORP,4,2,8,11,704
CSP-IMG,1,2,8,17,272


In [163]:
# SELECT count(*) as c2, count(DISTINCT browser_id), count(DISTINCT outcome_value) as c1, test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id 
# FROM "Result" GROUP by test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id order by c1 DESC, c2 DESC

# Currently ignore "org_scheme"
res = df.loc[df["org_scheme"] == "https"].groupby(["test_name", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])[["browser_id", "outcome_str"]].agg(["count", "nunique"])
display(res)
display(res.value_counts().to_frame())

browser_id  \
                                                                                                                                     count   
test_name                     org_host                    resp_scheme resp_host                       relation_info response_id              
accesswindow_direct           sub.headers.websec.saarland http        headers.webappsec.eu            direct        1524                16   
                                                                                                                    1525                16   
                                                                                                                    1526                16   
                                                                                                                    1527                16   
                                                                                                                    1528                16   
...                                                                                                                                    ...   
subresourceloadingCORP_object sub.headers.websec.saarland https       sub.sub.headers.websec.saarland direct        1907                16   
                                                                                                                    2113                16   
                                                                                                                    2114                16   
                                                                                                                    2115                16   
                                                                                                                    2116                16   

                                                                                                                                         \
                                                                                                                                nunique   
test_name                     org_host                    resp_scheme resp_host                       relation_info response_id           
accesswindow_direct           sub.headers.websec.saarland http        headers.webappsec.eu            direct        1524             10   
                                                                                                                    1525             10   
                                                                                                                    1526             10   
                                                                                                                    1527             10   
                                                                                                                    1528             10   
...                                                                                                                                 ...   
subresourceloadingCORP_object sub.headers.websec.saarland https       sub.sub.headers.websec.saarland direct        1907             10   
                                                                                                                    2113             10   
                                                                                                                    2114             10   
                                                                                                                    2115             10   
                                                                                                                    2116             10   

                                                                                                                                outcome_str  \
                                                 

count
(browser_id, count) (browser_id, nunique) (outcome_str, count) (outcome_str, nunique)       
16                  10                    16                   1                        5102
                                                               2                         368
                                                               3                         138
12                  10                    12                   1                          96
14                  8                     14                   1                          80

### Differences (currently only look at HTTPS):
- COOP (accesswindow_direct):
    - [ ] Edge + Firefox: redirect first
    - [ ] Brave + Chrome: COOP first?
-  Framing:
    -  [ ] framing_iframe_sandbox: CSP-FA *; Firefox allows, all other browsers deny?
- PP not supported in Firefox
- OAC not supported in Firefox
    - [ ] Also random/caching behavior?
- PerfAPI:
    - [ ] Chrome/Firefox have different rules when to create an entry at all?
- COEP (CORP):
    - [ ] Firefox incorrect behavior for same-origin (applies same-site?)
- CORP:
    - [ ] same-origin: Edge does not check the scheme?
    - [x] Firefox applies CORP on object tags (even though it should not!)
    - [ ] Some other CORP and/or COEP strange stuff (Firefox)
- Referrer Policy: different default in Brave (unsafe-url is never allowed)
- ...

In [170]:
test_name_seen = set()
for test_name, org_host, resp_scheme, resp_host, relation_info, response_id in res.loc[res[("outcome_str", "nunique")] != 1].index:
    # Only look at each feature group once
    if test_name in test_name_seen:
        continue
    test_name_seen.add(test_name)
    print(test_name)
    # no org_scheme at the moment
    rows = df.loc[df["org_scheme"] == "https"].loc[(df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id)]
    with pd.option_context("display.max_colwidth", 200):
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        display(rows.drop_duplicates(subset="outcome_str")[["browser_id", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host"]])
    

accesswindow_direct


count
outcome_str                                        name          
{'window.open.opener': 'null'}                     brave        8
                                                   chrome       4
{'window.open.opener': 'object "[object Window]"'} edge         2
                                                   firefox      2

,browser_id,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host
5794,13,{'window.open.opener': 'null'},"[[Cross-Origin-Opener-Policy, same-origin-allow-popups], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",accesswindow_direct,https,sub.headers.websec.saarland,http,direct,headers.webappsec.eu
23100,14,"{'window.open.opener': 'object ""[object Window]""'}","[[Cross-Origin-Opener-Policy, same-origin-allow-popups], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",accesswindow_direct,https,sub.headers.websec.saarland,http,direct,headers.webappsec.eu


framing_iframe


count
outcome_str     name          
message timeout brave        8
                chrome       4
message send    firefox      2
message timeout edge         2

...

count
outcome_str name          
load        brave        8
            chrome       4
error       firefox      2
load        edge         2

,browser_id,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host
5262,13,load,"[[Cross-Origin-Resource-Policy, same-origin]]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct,sub.headers.websec.saarland
22587,14,error,"[[Cross-Origin-Resource-Policy, same-origin]]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct,sub.headers.websec.saarland


subresourceloadingCORP_object


count
outcome_str name          
load        brave        8
            chrome       4
error       firefox      2
load        edge         2

,browser_id,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host
5339,13,load,"[[Cross-Origin-Resource-Policy, same-origin], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",subresourceloadingCORP_object,https,sub.headers.websec.saarland,https,direct,headers.webappsec.eu
22716,14,error,"[[Cross-Origin-Resource-Policy, same-origin], [location, https://sub.headers.websec.saarland/_hp/common/empty.html]]",subresourceloadingCORP_object,https,sub.headers.websec.saarland,https,direct,headers.webappsec.eu


# Crash analysis

### Exceptions thrown
- Fixed:
    - Only Firefox; both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Response_ids: 185, 186, 341, 342; these are the ones that have status code 302!
        - Reason seems to be that Firefox switches to another tab in these cases and thus the wait for crashes. Solution: forcefully switch back to the original tab before waiting!
    - Only Brave: both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Reason seems to be the tab switching, if deactivated no issue occurs

In [147]:
with open("/home/ubuntu/wpt/_hp/tools/crawler/desktop-selenium-2023-11-20 16:09:24.649008.log", "r") as f:
    crashes = []
    for l in f.readlines():
        if "649008: http" in l:
            match = re.match(".*(http|https)://.*browser_id=(\d+)&label=(.*)&first_id=(\d+).*", l)
            crashes.append([match[1], match[2], match[3], match[4]])

In [148]:
crashes = pd.DataFrame(crashes, columns=["scheme", "browser_id", "label", "response_id"]).drop_duplicates()
crashes

,scheme,browser_id,label,response_id
0,http,59,RP,172
2,http,59,RP,173
4,https,59,OAC,339
6,https,59,RP,181
8,http,75,RP,171
10,http,75,RP,172
12,https,75,OAC,330
14,https,75,OAC,338
16,https,75,RP,180
18,https,74,OAC,337
